In [95]:
import csv
import operator

In [96]:
def fuzzyFollowers (row):
    listFuzzy = []
    if row[1]<=20000:
        listFuzzy.append(["LOW",1])
    elif row[1]>20000 and row[1]<40000:
        listFuzzy.append(["LOW", (40000-row[1])/(40000-20000)])
        listFuzzy.append(["MEDIUM", (row[1]-20000)/(40000-20000)])
    elif row[1]>=40000 and row[1]<=60000:
        listFuzzy.append(["MEDIUM",1])
    elif row[1]>60000 and row[1]<80000:
        listFuzzy.append(["MEDIUM", (80000-row[1])/(80000-40000)])
        listFuzzy.append(["HIGH", (row[1]-40000)/(80000-40000)])
    elif row[1]>80000:
        listFuzzy.append(["HIGH",1])
    return listFuzzy
    

In [97]:
def fuzzyEngRate (row):
    listFuzzy = []
    if row[2]<=2.0:
        listFuzzy.append(["LOW",1])
    elif row[2]>2.0 and row[1]<4.0:
        listFuzzy.append(["LOW", (4.0-row[2])/(4.0-2.0)])
        listFuzzy.append(["MEDIUM", (row[2]-2.0)/(4.0-2.0)])
    elif row[2]>=4.0 and row[1]<=6.0:
        listFuzzy.append(["MEDIUM",1])
    elif row[2]>6.0 and row[1]<8.0:
        listFuzzy.append(["MEDIUM", (8.0-row[2])/(8.0-6.0)])
        listFuzzy.append(["HIGH", (row[2]-6.0)/(8.0-6.0)])
    elif row[2]>=8.0:
        listFuzzy.append(["HIGH",1])
    return listFuzzy

In [98]:
def inference (inFollower,inEngRate):
    arrInference = []
    for rowFollower in inFollower:
        for rowEngRate in inEngRate:
            if rowFollower[0] == "HIGH" and rowEngRate[0] == "HIGH":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Considered", rowFollower[1]])
                else:
                    arrInference.append(["Considered", rowEngRate[1]])
            elif rowFollower[0] == "HIGH" and rowEngRate[0] == "MEDIUM":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Rejected", rowFollower[1]])
                else:
                    arrInference.append(["Rejected", rowEngRate[1]])
            elif rowFollower[0] == "HIGH" and rowEngRate[0] == "LOW":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Rejected", rowFollower[1]])
                else:
                    arrInference.append(["Rejected", rowEngRate[1]])
            elif rowFollower[0] == "MEDIUM" and rowEngRate[0] == "HIGH":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Accepted", rowFollower[1]])
                else:
                    arrInference.append(["Accepted", rowEngRate[1]])
            elif rowFollower[0] == "MEDIUM" and rowEngRate[0] == "MEDIUM":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Considered", rowFollower[1]])
                else:
                    arrInference.append(["Considered", rowEngRate[1]])
            elif rowFollower[0] == "MEDIUM" and rowEngRate[0] == "LOW":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Rejected", rowFollower[1]])
                else:
                    arrInference.append(["Rejected", rowEngRate[1]])

            elif rowFollower[0] == "LOW" and rowEngRate[0] == "MEDIUM":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Accepted", rowFollower[1]])
                else:
                    arrInference.append(["Accepted", rowEngRate[1]])
            elif rowFollower[0] == "LOW" and rowEngRate[0] == "HIGH":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Accepted", rowFollower[1]])
                else:
                    arrInference.append(["Accepted", rowEngRate[1]])
            elif rowFollower[0] == "LOW" and rowEngRate[0] == "LOW":
                if rowFollower[1] < rowEngRate[1]:
                    arrInference.append(["Considered", rowFollower[1]])
                else:
                    arrInference.append(["Considered", rowEngRate[1]])
    if len(arrInference)>1:
        if arrInference[0][0] == "Accepted" and arrInference[1][0] == "Accepted":
            if arrInference[0][1] > arrInference[1][1]:
                del arrInference[1]
            else:
                del arrInference[0]
        elif arrInference[0][0] == "Considered" and arrInference[1][0] == "Considered":
            if arrInference[0][1] > arrInference[1][1]:
                del arrInference[1]
            else:
                del arrInference[0]
        elif arrInference[0][0] == "Rejected" and arrInference[1][0] == "Rejected":
            if arrInference[0][1] > arrInference[1][1]:
                del arrInference[1]
            else:
                del arrInference[0]
    return arrInference

In [99]:
def deffuzification (arrInference):
    x = 0
    y = 0
    for inInfer in arrInference:
        if inInfer[0]=="Accepted":
            x = x+(inInfer[1]*90)
            y = y+(inInfer[1])
        elif inInfer[0]=="Considered":
            x = x+(inInfer[1]*60)
            y = y+(inInfer[1])
        elif inInfer[0]=="Rejected":
            x = x+(inInfer[1]*40)
            y = y+(inInfer[1])

    try:
        score = x / y
    except ZeroDivisionError:
        score = 0
        
    return score

In [100]:
with open('datainfluencer.csv') as filecsv:
    readcsv = csv.reader(filecsv,skipinitialspace = True)
    listData = []
    next(readcsv)
    chosen = []
    data = []
    for row in readcsv:
        row[1] = float(row[1])
        row[2] = float(row[2])
        row[0] = int(row[0])
        listData.append([row[0],deffuzification(inference(fuzzyFollowers(row),fuzzyEngRate(row)))])
#     print(listData,"/n"

    data = sorted(listData, key=operator.itemgetter(1), reverse=True)
    for i in range(0, 20):
        chosen.append([data[i][0]])

    terpilih = sorted(chosen, key=operator.itemgetter(0))
#     print(data,"\n")
    print("20 Influencer Terpilih:\n")
    print(terpilih,"\n")
    print("Naufal Hilmiaji - IF4108 - 1301174314")
    
    with open('chosen.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(chosen)

20 Influencer Terpilih:

[[1], [3], [9], [17], [22], [24], [29], [30], [31], [39], [40], [44], [45], [55], [57], [58], [64], [77], [82], [83]] 

Naufal Hilmiaji - IF4108 - 1301174314
